In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [10]:
%%writefile vector.cu
#include<bits/stdc++.h>
#include<cstdlib>
using namespace std;
#define N 512

__global__ void add(int *a,int *b,int *c){
  int idx = blockDim.x + blockIdx.x + threadIdx.x;
  if(idx<N)
  {
    c[idx] = a[idx] + b[idx];
  }
}

int main(){
  int *a,*b,*c;
  int *da,*db,*dc;
  int size = N*sizeof(int);

  a = (int *)malloc(size);
  b = (int *)malloc(size);
  c = (int *)malloc(size);

  cudaMalloc((void **)&da,size);
  cudaMalloc((void **)&db,size);
  cudaMalloc((void **)&dc,size);

  for(int i=0;i<N;i++){
    a[i]=i;
    b[i]=i*2;
  }

  cudaMemcpy(da,a,size,cudaMemcpyHostToDevice);
  cudaMemcpy(db,b,size,cudaMemcpyHostToDevice);
  cudaMemcpy(dc,c,size,cudaMemcpyHostToDevice);

  add<<<N,1>>>(da,db,dc);

  cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

  cout <<"c[0] = "<<c[0]<<" ,c["<<N-1<<"] = "<<c[N-1];
  free(a);
  free(b);
  free(c);

  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);

  return 0;

}

Overwriting vector.cu


In [11]:
!nvcc vector.cu -o vect

In [12]:
!./vect


c[0] = 0 ,c[511] = 1533

In [13]:
!nvprof ./vect

==1176== NVPROF is profiling process 1176, command: ./vect
==1176== Profiling application: ./vect
c[0] = 0 ,c[511] = 1533==1176== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   44.06%  5.6960us         1  5.6960us  5.6960us  5.6960us  add(int*, int*, int*)
                   38.86%  5.0240us         3  1.6740us  1.5360us  1.9520us  [CUDA memcpy HtoD]
                   17.09%  2.2090us         1  2.2090us  2.2090us  2.2090us  [CUDA memcpy DtoH]
      API calls:   99.38%  220.51ms         3  73.502ms  2.4730us  220.50ms  cudaMalloc
                    0.45%  1.0033ms         1  1.0033ms  1.0033ms  1.0033ms  cuDeviceGetPCIBusId
                    0.06%  123.20us       101  1.2190us     125ns  50.667us  cuDeviceGetAttribute
                    0.04%  98.999us         3  32.999us  2.5760us  87.428us  cudaFree
                    0.04%  80.165us         4  20.041us  8.3570us  40.614us  cudaMemcpy
                    0.